In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, PCA
from sklearn.model_selection import train_test_split

In [2]:
client_id = "53e3e2d67bad4559a0c98aa634b14a9b"
client_secret = "4e8a81a322ef4d129d811e1e0a6986b4"

In [3]:
import requests
import json
import pandas as pd

# Define the search query parameters
genres = ["pop", "rock", "hip hop", "Electronic","Country","R&B","Jazz","Classical"]


# Set up the API request headers
auth_response = requests.post("https://accounts.spotify.com/api/token", {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
})
access_token = auth_response.json()["access_token"]
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Retrieve track data for each genre
tracks_data = []
for genre in genres:
    search_response = requests.get(f"https://api.spotify.com/v1/search?q=genre:{genre}&type=track&market=US&limit=50", headers=headers)
    if search_response.status_code != 200:
        print(f"Error searching for {genre} tracks:", search_response.json())
    else:
        tracks = search_response.json()["tracks"]["items"]
        for track in tracks:
            track_id = track["id"]
            track_response = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers)
            if track_response.status_code != 200:
                print(f"Error getting {track_id} track info:", track_response.json())
            else:
                track_data = track_response.json()
                title = track_data["name"]
                artist = track_data["artists"][0]["name"]
                album = track_data["album"]["name"]
                popularity = (100-track_data["popularity"])/100
                duration = track['duration_ms'] / 1000
                ex = int(track['explicit'])
                g=genre
                tracks_data.append({
                    "title": title,
                    "artist": artist,
                    "album": album,
                    "popularity": popularity,
                    "genres": g,
                    "explicit":ex,
                    "duration":duration
                })

# Create a Pandas DataFrame from the track data
df = pd.DataFrame(tracks_data)
display(df)


,title,artist,album,popularity,genres,explicit,duration
0,Flowers,Miley Cyrus,Flowers,0.00,pop,0,200.454
1,Kill Bill,SZA,SOS,0.08,pop,0,153.946
2,Kill Bill,SZA,SOS,0.08,pop,0,153.946
3,Sure Thing,Miguel,All I Want Is You,0.10,pop,0,195.373
4,Rich Flex,Drake,Her Loss,0.11,pop,1,239.359
...,...,...,...,...,...,...,...
395,Handel / Orch. Hale: Keyboard Suite No. 4 in D...,George Frideric Handel,Handel Sarabande,0.36,Classical,0,207.266
396,"Symphony No. 25 in G Minor, K. 183: I. Allegro...",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,460.026
397,"Die Zauberflöte, K.620 / Act 2: Marsch der Pri...",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,149.053
398,"Allegro in D Major, K. 626b/16",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,101.279


In [4]:
print(track_data.keys())

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


In [5]:
duplicates = df.duplicated()
print(df[duplicates])

         title artist album  popularity genres  explicit  duration
2    Kill Bill    SZA   SOS        0.08    pop         0   153.946
253  Kill Bill    SZA   SOS        0.08    R&B         0   153.946


In [6]:
df = df.drop_duplicates().reset_index(drop=True)

In [16]:
df

,title,artist,album,popularity,genres,explicit,duration
0,Flowers,Miley Cyrus,Flowers,0.00,pop,0,200.454
1,Kill Bill,SZA,SOS,0.08,pop,0,153.946
2,Sure Thing,Miguel,All I Want Is You,0.10,pop,0,195.373
3,Rich Flex,Drake,Her Loss,0.11,pop,1,239.359
4,Romantic Homicide,d4vd,Romantic Homicide,0.11,pop,0,132.630
...,...,...,...,...,...,...,...
393,Handel / Orch. Hale: Keyboard Suite No. 4 in D...,George Frideric Handel,Handel Sarabande,0.36,Classical,0,207.266
394,"Symphony No. 25 in G Minor, K. 183: I. Allegro...",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,460.026
395,"Die Zauberflöte, K.620 / Act 2: Marsch der Pri...",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,149.053
396,"Allegro in D Major, K. 626b/16",Wolfgang Amadeus Mozart,Best of Klassik,1.00,Classical,0,101.279


In [14]:
df = df[df['']==0]